In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D
from CGA.cluster_filters import cluster_filters

In [2]:
model = load_model('../Models/NN/model_renet50.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_2 (Dense)             (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [3]:
resnet50_layers = model.layers[0].layers
for i, warstwa in enumerate(resnet50_layers):
    print(f'Warstwa ResNet50 {i + 1}: {warstwa.name}, Typ: {warstwa.__class__.__name__}, Shape: {warstwa.output_shape}')


Warstwa ResNet50 1: input_3, Typ: InputLayer, Shape: [(None, None, None, 3)]
Warstwa ResNet50 2: conv1_pad, Typ: ZeroPadding2D, Shape: (None, None, None, 3)
Warstwa ResNet50 3: conv1_conv, Typ: Conv2D, Shape: (None, None, None, 64)
Warstwa ResNet50 4: conv1_bn, Typ: BatchNormalization, Shape: (None, None, None, 64)
Warstwa ResNet50 5: conv1_relu, Typ: Activation, Shape: (None, None, None, 64)
Warstwa ResNet50 6: pool1_pad, Typ: ZeroPadding2D, Shape: (None, None, None, 64)
Warstwa ResNet50 7: pool1_pool, Typ: MaxPooling2D, Shape: (None, None, None, 64)
Warstwa ResNet50 8: conv2_block1_1_conv, Typ: Conv2D, Shape: (None, None, None, 64)
Warstwa ResNet50 9: conv2_block1_1_bn, Typ: BatchNormalization, Shape: (None, None, None, 64)
Warstwa ResNet50 10: conv2_block1_1_relu, Typ: Activation, Shape: (None, None, None, 64)
Warstwa ResNet50 11: conv2_block1_2_conv, Typ: Conv2D, Shape: (None, None, None, 64)
Warstwa ResNet50 12: conv2_block1_2_bn, Typ: BatchNormalization, Shape: (None, None, None,

In [4]:
resnet_model = model.get_layer('resnet50')

output_layer = 'conv4_block1_2_conv'

layer = resnet_model.get_layer(output_layer)
weights = layer.get_weights()[0]
biases = layer.get_weights()[1] 

In [5]:
weights.shape

(3, 3, 256, 256)

In [6]:
biases.shape

(256,)

In [7]:
num_filters_to_remove = 100

filters_to_remove = np.random.choice(weights.shape[3], num_filters_to_remove, replace=False)

new_weights = np.delete(weights, filters_to_remove, axis=3)
new_biases = np.delete(biases, filters_to_remove)

In [8]:
def prune_filter(original_model, cut_off_layer_name, indexes):
    # Warswtwa którą chcę przyciąć
    layer_to_prune = resnet_model.get_layer(cut_off_layer_name)
    
    all_layers = original_model.layers

    # Ideks warstwy do przycięcia
    layer_to_prune_index = all_layers.index(layer_to_prune)

    # Blok poprzedzający przycinany blok
    previous_layer = next((layer for layer in all_layers[:layer_to_prune_index][::-1] if layer.name.endswith("out")), None)
    # Następny blok po przycinanym bloku
    next_layer = next((layer for layer in all_layers[layer_to_prune_index + 1:] if layer.name.endswith("add")), None)

    # Tworzę 3 modele, które później złącze ze sobą, prev i nex nie będzie zmieniany a model_to_prune będzie przycięty
    model_prev = Model(inputs=original_model.input, outputs=previous_layer.output)        
    model_to_prune = Model(inputs=original_model.layers[all_layers.index(previous_layer) + 1].input, outputs=next_layer.output)       
    model_next = Model(inputs=original_model.layers[all_layers.index(next_layer) + 1].input, outputs=original_model.output) 

    combined_input = Input(shape=(32, 32, 3))
            
    output_prev = model_prev(combined_input)
    
    # Kopiuję konfiguracje warstwy do przycięcia
    config = layer_to_prune.get_config()
    config['filters'] = len(indexes)

    # Zamiast przycinania strej warstwy tworze nową z tą samą konfiguracja, wagami i biasem
    new_layer = Conv2D(**config)
    input_shape = layer_to_prune.input_shape
    new_layer.build(input_shape)

    # Ustawianie wag i biasów
    weights = layer_to_prune.get_weights()[0][:, :, :, indexes]
    if len(layer_to_prune.get_weights()) > 1:
        biases = layer_to_prune.get_weights()[1][indexes]
        new_weights = [weights, biases]
    else:
        new_weights = [weights]
    new_layer.set_weights(new_weights)
    
    for layer_prune in model_to_prune.layers:
        if layer_prune.name != cut_off_layer_name:
            if layer_prune.name.endswith("_conv"):
                output_prev = Conv2D(filters=layer_prune.input_shape[-1], kernel_size=(1, 1))(output_prev)
            if layer_prune.name.endswith("_add"):
                output_prev = layer_prune([output_prev])
            else:
                output_prev = layer_prune(output_prev)
        else:
            # output_prev = new_layer(output_prev)
            output_prev = layer_prune(output_prev)
            output_prev = Conv2D(filters=256, kernel_size=(1, 1), name='adaptation_conv')(output_prev)

    # output_to_prune = model_to_prune(output_prev)
    final_output = model_next(output_prev)
    # print(final_output.layers)

    # Na nowo łaczę przycięty model
    combined_model = Model(inputs=combined_input, outputs=final_output)

    # combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return combined_model


In [9]:
weights.shape

(3, 3, 256, 256)

In [10]:
# weights_list = np.array([weights[:, :, :, i] for i in range(weights.shape[-1])])
weights_list = weights.reshape(256, -1)
weights_list.shape
# filter_indexes = cluster_filters(weights_list)

(256, 2304)

In [11]:
# from sklearn.cluster import KMeans
# 
# k = 100  # Liczba klastrów
# knn = KMeans(n_clusters=k)
# knn.fit(weights_list)
# klastry = knn.labels_
# 
# indexes = []
# for i in range(k):
#     indeksy_klastra = np.where(klastry == i)[0]
#     losowy_indeks = np.random.choice(indeksy_klastra)
#     indexes.append(losowy_indeks)
# 
# pruned_model = prune_filter(resnet_model, output_layer, indexes)

In [12]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.9, min_samples=1)
dbscan.fit(weights_list)


cluster_indices = np.unique(dbscan.labels_)

indexes = []
for cluster_index in cluster_indices:
    cluster_points = np.where(dbscan.labels_ == cluster_index)[0]
    index = np.random.choice(cluster_points)
    indexes.append(index)
    
pruned_model = prune_filter(resnet_model, output_layer, indexes)

In [13]:
# import random
# 
# indexes = [random.randint(0, 255) for _ in range(100)]
# pruned_model = prune_filter(resnet_model, output_layer, indexes)

In [14]:
from keras import Sequential

prune_input = Input(shape=(32, 32, 3))
# prune_output = pruned_model(prune_input)
# new_output = model.layers[1](prune_output)
# new_model = Model(inputs=prune_input, outputs=new_output)
new_model = Sequential()
new_model.add(prune_input)
new_model.add(pruned_model)
new_model.add(model.layers[1])

In [15]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train_X = np.expand_dims(train_X, axis=-1)  # Add a channel dimension for ResNet50
train_X = np.repeat(train_X, 3, axis=-1)  # Repeat the single channel to simulate 3 channels
train_X = np.pad(train_X, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')  # Resize to 224x224
train_y = to_categorical(train_y, 10)
new_model.fit(x = train_X, y= train_y, epochs=1)

1875/1875 [==============================] - 1698s 904ms/step - loss: 0.3518 - accuracy: 0.9287


In [16]:
test_X = np.expand_dims(test_X, axis=-1)
test_X = np.repeat(test_X, 3, axis=-1)
test_X = np.pad(test_X, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')
result = new_model.evaluate(test_X, to_categorical(test_y, 10))

313/313 [==============================] - 24s 73ms/step - loss: 0.0883 - accuracy: 0.9753


In [17]:
new_model.save('../NN/pruned_DBSCAN_model_mnist_renet50.h5')

In [20]:
model_final = load_model('../NN/pruned_DBSCAN_model_mnist_renet50.h5')
# test_X = np.expand_dims(test_X, axis=-1)  # Add a channel dimension for ResNet50
# test_X = np.repeat(test_X, 3, axis=-1)  # Repeat the single channel to simulate 3 channels
# test_X = np.pad(test_X, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')  # Resize to 224x224
# result = model_final.evaluate(test_X, to_categorical(test_y, 10))
# print(result)

ValueError: A merge layer should be called on a list of inputs. Received: input_shape=(None, None, None, 1024) (not a list of shapes)